# Outcome Critical Statistics 

## Loading Libraries

In [1]:
import pandas as pd
import os
import time
import random
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model
from tabulate import tabulate

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

confidence_interval = 95
random_seed=0

2025-03-16 16:45:27.857072: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-16 16:45:27.875381: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-16 16:45:27.885045: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-16 16:45:27.887746: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-16 16:45:27.895964: I tensorflow/core/platform/cpu_feature_guar

## Summary Statistics

In [14]:
df_master = pd.read_csv('df_critical_categorical_variables_15_3_Latest.csv')

In [3]:
from sklearn.preprocessing import LabelEncoder

# Fit and transform to get encoded values
encoder = LabelEncoder()
df_master['arrival_transport_encoded'] = encoder.fit_transform(df_master['arrival_transport'])

# see the classes
print("Classes found by LabelEncoder:", encoder.classes_)

Classes found by LabelEncoder: ['AMBULANCE' 'HELICOPTER' 'OTHER' 'UNKNOWN' 'WALK IN']


In [7]:
import pandas as pd

# List of categorical variables (n=13)
categorical_vars = [
    'triage_acuity_iter_category',
    'arrival_transport',
    'score_NEWS_category',
    'triage_sbp_iter_category',
    'med_event',
    'score_CART_category',
    'score_CCI_category',
    'n_icu_365d_category',
    'triage_heartrate_iter_category',
    'triage_resprate_iter_category',
    'age_category',
    'triage_dbp_iter_category',
    'triage_pain_iter_category',
]


# Initialize an empty list to store the table data
table_data = []

# Loop through categorical variables to calculate counts and percentages
for variable in categorical_vars:
    # Get the unique categories for the variable
    categories = df_master[variable].dropna().unique()
    
    for category in categories:
        total_count = df_master[df_master[variable] == category].shape[0]

        # Critical outcome cases
        critical_count = df_master[(df_master[variable] == category) & (df_master['outcome_critical'] == 1)].shape[0]
        critical_percentage = (critical_count / total_count) * 100 if total_count > 0 else 0

        # Non-critical outcome cases
        non_critical_count = df_master[(df_master[variable] == category) & (df_master['outcome_critical'] == 0)].shape[0]
        non_critical_percentage = (non_critical_count / total_count) * 100 if total_count > 0 else 0

        # Store the results in a list
        table_data.append([
            variable, category, total_count,
            critical_count, f"{critical_percentage:.2f}%",
            non_critical_count, f"{non_critical_percentage:.2f}%"
        ])

# Create a DataFrame for better visualization
df_summary = pd.DataFrame(table_data, columns=[
    "Variable", "Category", "Total Count", 
    "Critical Count", "Critical Percentage",
    "Non-Critical Count", "Non-Critical Percentage"
])


In [8]:
df_summary

,Variable,Category,Total Count,Critical Count,Critical Percentage,Non-Critical Count,Non-Critical Percentage
0,triage_acuity_iter_category,ESI 3 - Urgent,229140,5017,2.19%,224123,97.81%
1,triage_acuity_iter_category,ESI 2 - Emergent,141506,14229,10.06%,127277,89.94%
2,triage_acuity_iter_category,ESI 4 - Less Urgent,28515,13,0.05%,28502,99.95%
3,triage_acuity_iter_category,ESI 1 - Resuscitation,24826,8911,35.89%,15915,64.11%
4,triage_acuity_iter_category,ESI 5 - Non-Urgent,1100,0,0.00%,1100,100.00%
5,arrival_transport,AMBULANCE,155752,18328,11.77%,137424,88.23%
6,arrival_transport,WALK IN,251849,5330,2.12%,246519,97.88%
7,arrival_transport,UNKNOWN,15352,3854,25.10%,11498,74.90%
8,arrival_transport,OTHER,1266,56,4.42%,1210,95.58%
9,arrival_transport,HELICOPTER,868,602,69.35%,266,30.65%


In [21]:
#df_master = pd.read_csv('df_critical_categorical_variables.csv')

In [ ]:
### Overall

In [28]:
df_master = pd.read_csv('df_master_ready_critical.csv')     #for the percent/count

In [29]:
df_master

,index,subject_id,hadm_id,stay_id,intime,outtime,gender,race,arrival_transport,disposition,anchor_age,anchor_year,dod,admittime,dischtime,deathtime,ethnicity,edregtime,edouttime,insurance,in_year,age,outcome_inhospital_mortality,ed_los,intime_icu,time_to_icu_transfer,outcome_icu_transfer_12h,outcome_hospitalization,outcome_critical,n_ed_30d,n_ed_90d,n_ed_365d,next_ed_visit_time,next_ed_visit_time_diff,outcome_ed_revisit_3d,n_hosp_30d,n_hosp_90d,n_hosp_365d,n_icu_30d,n_icu_90d,n_icu_365d,ed_los_hours,time_to_icu_transfer_hours,next_ed_visit_time_diff_days,triage_temperature,triage_heartrate,triage_resprate,triage_o2sat,triage_sbp,triage_dbp,...,eci_Coagulopathy,eci_Obesity,eci_WeightLoss,eci_FluidsLytes,eci_BloodLoss,eci_Anemia,eci_Alcohol,eci_Drugs,eci_Psychoses,eci_Depression,ed_temperature_last,ed_heartrate_last,ed_resprate_last,ed_o2sat_last,ed_sbp_last,ed_dbp_last,ed_pain_last,n_med,n_medrecon,time_to_microbio_event_hrs,microbiology_event,time_to_med_event_hr,med_event,triage_temperature_iter,triage_heartrate_iter,triage_resprate_iter,triage_o2sat_iter,triage_sbp_iter,triage_dbp_iter,triage_pain_iter,ed_temperature_last_iter,ed_heartrate_last_iter,ed_resprate_last_iter,ed_o2sat_last_iter,ed_sbp_last_iter,ed_dbp_last_iter,ed_pain_last_iter,triage_acuity_iter,score_CCI,triage_MAP,score_REMS,score_CART,score_NEWS,score_NEWS2,score_MEWS,arrival_transport_AMBULANCE,arrival_transport_HELICOPTER,arrival_transport_OTHER,arrival_transport_UNKNOWN,arrival_transport_WALK_IN
0,0,10000032,22595853.0,33258284,2180-05-06 19:17:00,2180-05-06 23:30:00,0,WHITE,AMBULANCE,ADMITTED,52,2180,2180-09-09,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,Other,2180,52,False,0 days 04:13:00,NaN,NaN,False,True,False,0,0,0,2180-06-26 15:54:00,50 days 16:24:00,False,0,0,0,0,0,0,4.22,NaN,50.68,36.888889,70.0,16.0,97.0,106.0,63.0,...,0,0,0,0,0,0,0,0,0,0,36.500000,79.0,16.0,98.0,107.0,60.0,0.0,0.0,9.0,10.92,1.0,2.37,1.0,37.0,70.0,16.0,97.0,106.0,63.0,0.0,36.500000,79.0,16.0,98.000000,107.0,60.0,0.000000,3.0,1,77.333333,2,0,1,1,1,True,False,False,False,False
1,1,10000032,22841357.0,38112554,2180-06-26 15:54:00,2180-06-26 21:31:00,0,WHITE,AMBULANCE,ADMITTED,52,2180,2180-09-09,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,Medicaid,2180,52,False,0 days 05:37:00,NaN,NaN,False,True,False,0,1,1,2180-07-22 16:24:00,25 days 18:53:00,False,0,1,1,0,0,0,5.62,NaN,25.79,37.166667,88.0,18.0,97.0,116.0,88.0,...,0,0,0,0,0,0,0,0,0,1,36.611111,86.0,17.0,93.0,96.0,57.0,5.0,3.0,12.0,6.80,1.0,3.33,1.0,37.0,88.0,18.0,97.0,116.0,88.0,10.0,36.611111,86.0,17.0,93.000000,96.0,57.0,5.000000,3.0,6,97.333333,2,0,0,0,1,True,False,False,False,False
2,3,10000032,29079034.0,32952584,2180-07-22 16:24:00,2180-07-23 05:54:00,0,WHITE,AMBULANCE,HOME,52,2180,2180-09-09,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,Medicaid,2180,52,False,0 days 13:30:00,2180-07-23 14:00:00,0 days 08:06:00,True,True,True,1,2,2,2180-07-23 05:54:00,0 days 00:00:00,True,1,2,2,0,0,0,13.50,8.1,0.00,36.555556,87.0,14.0,97.0,71.0,43.0,...,1,0,0,1,0,0,0,0,0,1,36.777778,85.0,18.0,98.0,81.0,38.0,0.0,2.0,14.0,13.68,1.0,11.53,1.0,37.0,87.0,14.0,97.0,71.0,43.0,7.0,36.777778,85.0,18.0,98.000000,81.0,38.0,0.000000,2.0,6,52.333333,4,4,3,3,2,True,False,False,False,False
3,4,10000032,29079034.0,39399961,2180-07-23 05:54:00,2180-07-23 14:00:00,0,WHITE,AMBULANCE,ADMITTED,52,2180,2180-09-09,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,Medicaid,2180,52,False,0 days 08:06:00,2180-07-23 14:00:00,0 days 00:00:00,True,True,True,2,3,3,2180-08-05 20:58:00,13 days 06:58:00,False,1,2,2,0,0,0,8.10,0.0,13.29,37.055556,77.0,16.0,98.0,96.0,50.0,...,1,0,0,1,0,0,0,0,0,1,37.222222,96.0,18.0,97.0,86.0,45.0,NaN,8.0,14.0,13.68,1.0,1.05,1.0,37.0,77.0,16.0,98.0,96.0,50.0,-0.0,37.222222,96.0,18.0,97.000000,86.0,45.0,0.594541,2.0,6,65.333333,4,0,2,2,2,True,False,False,False,False
4,2,10

In [32]:
# Total number of records
total_count = len(df_master)



# Count and percentage of Critical outcome
outcome_critical = df_master['outcome_critical'].sum()  # True values count
outcome_critical_percent = (outcome_critical / total_count) * 100


# Count and percentage of ICU revisits within 12 hrs
icu_revisit_count = df_master['outcome_icu_transfer_12h'].sum()  # True values count
icu_revisit_percent = (icu_revisit_count / total_count) * 100

# Count and percentage of in-hospital deaths
death_count = df_master['outcome_inhospital_mortality'].sum()  # True values count
death_percent = (death_count / total_count) * 100

# Remaining observations for ICU revisits within 12 hrs
remaining_icu_revisit_count = total_count - icu_revisit_count
remaining_icu_revisit_percent = 100 - icu_revisit_percent

# Remaining observations for in-hospital deaths
remaining_death_count = total_count - outcome_critical
remaining_death_percent = 100 - outcome_critical_percent

# Total count of remaining observations
remaining_total_count = total_count - outcome_critical
remaining_total_percent = 100 - outcome_critical_percent

# Output results
print(f"Total records: {total_count}")
print(f"Critical Outcome: {outcome_critical} ({outcome_critical_percent:.2f}%)")
print(f"ICU Revisits within 12 hrs: {icu_revisit_count} ({icu_revisit_percent:.2f}%)")
print(f"In-hospital Deaths: {death_count} ({death_percent:.2f}%)")
print(f"Remaining Observations: {remaining_total_count} ({remaining_total_percent:.2f}%)")


Total records: 425087
Critical Outcome: 28170 (6.63%)
ICU Revisits within 12 hrs: 26589 (6.25%)
In-hospital Deaths: 4748 (1.12%)
Remaining Observations: 396917 (93.37%)


In [35]:
# Count and percentage of cases with both ICU transfer within 12 hours and in-hospital mortality
both_count = ((df_master['outcome_icu_transfer_12h'] == 1) & 
              (df_master['outcome_inhospital_mortality'] == 1)).sum()
both_percent = (both_count / total_count) * 100
print(f"Cases with BOTH ICU Transfer within 12 hrs & In-hospital Mortality: {both_count} ({both_percent:.2f}%)")


Cases with BOTH ICU Transfer within 12 hrs & In-hospital Mortality: 3167 (0.75%)


In [36]:
(26589 + 4748)-3167

28170

# Chi square Test (testing statistical significance)

In [ ]:
df_master = pd.read_csv('df_critical_categorical_variables_15_3_Latest.csv')

In [12]:
import pandas as pd
from scipy.stats import chi2_contingency

# List of categorical variables to test
categorical_vars = [
    'triage_acuity_iter_numeric',
    'arrival_transport_encoded',
    'score_NEWS_numeric',
    'triage_sbp_iter_numeric',
    'med_event',
    'score_CART_numeric',
    'score_CCI_numeric',
    'n_icu_365d_numeric',
    'triage_heartrate_iter_numeric',
    'triage_resprate_iter_numeric',
    'age_numeric',
    'triage_dbp_iter_numeric',
    'triage_pain_iter_numeric']
    #'score_NEWS2_numeric',
    #'n_ed_365d_numeric'


# Initialize a list to collect results
results = []

# Loop over each categorical variable and perform chi-square test
for var in categorical_vars:
    contingency_table = pd.crosstab(df_master[var], df_master['outcome_critical'])
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    results.append({'Variable': var, 'Chi2': chi2, 'p-value': p, 'Degrees of Freedom': dof})
    print(f"{var}: chi2 = {chi2:.4f}, p-value = {p:.4f}, dof = {dof}")

# Convert results into a DataFrame for easy viewing
results_df = pd.DataFrame(results)
print(results_df)


triage_acuity_iter_numeric: chi2 = 46420.0247, p-value = 0.0000, dof = 4
arrival_transport_encoded: chi2 = 28932.3392, p-value = 0.0000, dof = 4
score_NEWS_numeric: chi2 = 29059.5125, p-value = 0.0000, dof = 4
triage_sbp_iter_numeric: chi2 = 10261.3489, p-value = 0.0000, dof = 4
med_event: chi2 = 429.6625, p-value = 0.0000, dof = 1
score_CART_numeric: chi2 = 26762.3386, p-value = 0.0000, dof = 4
score_CCI_numeric: chi2 = 9810.8625, p-value = 0.0000, dof = 4
n_icu_365d_numeric: chi2 = 6854.4056, p-value = 0.0000, dof = 3
triage_heartrate_iter_numeric: chi2 = 10759.6808, p-value = 0.0000, dof = 4
triage_resprate_iter_numeric: chi2 = 16124.8546, p-value = 0.0000, dof = 4
age_numeric: chi2 = 9914.1591, p-value = 0.0000, dof = 4
triage_dbp_iter_numeric: chi2 = 3171.3224, p-value = 0.0000, dof = 4
triage_pain_iter_numeric: chi2 = 1414.6003, p-value = 0.0000, dof = 3
                         Variable          Chi2        p-value  \
0      triage_acuity_iter_numeric  46420.024689   0.000000e+0

## Generate Latex Tables Code

In [20]:
# import matplotlib.pyplot as plt


# Initialize first table
latex_table_part1 = """
\\begin{table}[h]
\\centering
\\renewcommand{\\arraystretch}{1.2}
\\setlength{\\tabcolsep}{3pt}
\\begin{threeparttable}
\\begin{tabular}{|p{3.0cm}|p{2.05cm}|p{2.05cm}|p{0.9cm}|}
\\hline
\\textbf{Variable} & \\textbf{Critical Outcome} & \\textbf{Non-Critical Outcome} & \\textbf{GB Importance} \\\\ 
\\hline
"""

current_variable = None
split_point_reached = False  # Flag to track when to split the table

for _, row in df_summary.iterrows():
    variable, category, total_count, crit_count, crit_percent, non_crit_count, non_crit_percent = row

    # If new categorical variable, add header
    if variable != current_variable:
        if current_variable is not None:
            latex_table_part1 += "\\hline\n"  # Close previous section

        # Manually split table when "Triage Resprate Iter Category" is reached
        if variable == "Triage Resprate Iter Category":
            split_point_reached = True
            break  # Stop adding to the first table

        latex_table_part1 += f"\\multicolumn{{4}}{{|l|}}{{\\textbf{{{variable.replace('_', ' ').title()}}}}} \\\\ \\hline\n"
        current_variable = variable

    # Ensure proper formatting of percentage values (removing extra % signs)
    crit_percent_str = f"{crit_percent:.2f}" if isinstance(crit_percent, (int, float)) else crit_percent.replace("%", "")
    non_crit_percent_str = f"{non_crit_percent:.2f}" if isinstance(non_crit_percent, (int, float)) else non_crit_percent.replace("%", "")

    # Add category values with properly escaped % symbols
    latex_table_part1 += "\\textit{{{}}} & {}\\% ({}) & {}\\% ({}) &  \\\\ \n".format(
        category, crit_percent_str, crit_count, non_crit_percent_str, non_crit_count
    )

# Close first table
latex_table_part1 += """\\hline
\\end{tabular}
\\begin{tablenotes}
\\item \\footnotesize All variables were statistically significant ($p<0.01$).
\\end{tablenotes}
\\end{threeparttable}
\\caption{Summary Statistics of Categorical Variables}
\\label{tab:summary_variables}
\\end{table}
"""

# Initialize second table for the remaining variables
latex_table_part2 = """
\\begin{table}[h]
\\centering
\\renewcommand{\\arraystretch}{1.2}
\\setlength{\\tabcolsep}{3pt}
\\begin{threeparttable}
\\begin{tabular}{|p{3.0cm}|p{2.05cm}|p{2.05cm}|p{0.9cm}|}
\\hline
\\textbf{Variable} & \\textbf{Critical Outcome} & \\textbf{Non-Critical Outcome} & \\textbf{GB Importance} \\\\ 
\\hline
"""

# Continue adding remaining variables to part 2
if split_point_reached:
    for _, row in df_summary.iterrows():
        variable, category, total_count, crit_count, crit_percent, non_crit_count, non_crit_percent = row

        # If new categorical variable, add header
        if variable != current_variable:
            if current_variable is not None:
                latex_table_part2 += "\\hline\n"  # Close previous section

            latex_table_part2 += f"\\multicolumn{{4}}{{|l|}}{{\\textbf{{{variable.replace('_', ' ').title()}}}}} \\\\ \\hline\n"
            current_variable = variable

        # Ensure proper formatting of percentage values (removing extra % signs)
        crit_percent_str = f"{crit_percent:.2f}" if isinstance(crit_percent, (int, float)) else crit_percent.replace("%", "")
        non_crit_percent_str = f"{non_crit_percent:.2f}" if isinstance(non_crit_percent, (int, float)) else non_crit_percent.replace("%", "")

        # Add category values with properly escaped % symbols
        latex_table_part2 += "\\textit{{{}}} & {}\\% ({}) & {}\\% ({}) &  \\\\ \n".format(
            category, crit_percent_str, crit_count, non_crit_percent_str, non_crit_count
        )

# Close second table
latex_table_part2 += """\\hline
\\end{tabular}
\\begin{tablenotes}
\\item \\footnotesize All variables were statistically significant ($p<0.01$).
\\end{tablenotes}
\\end{threeparttable}
\\caption{Summary Statistics of Categorical Variables (Continued)}
\\label{tab:summary_variables_continued}
\\end{table}
"""

# Output the two LaTeX table codes
print(latex_table_part1)
print("\n\\clearpage\n")  # Add a clear page before part 2
print(latex_table_part2)



\begin{table}[h]
\centering
\renewcommand{\arraystretch}{1.2}
\setlength{\tabcolsep}{3pt}
\begin{threeparttable}
\begin{tabular}{|p{3.0cm}|p{2.05cm}|p{2.05cm}|p{0.9cm}|}
\hline
\textbf{Variable} & \textbf{Critical Outcome} & \textbf{Non-Critical Outcome} & \textbf{GB Importance} \\ 
\hline
\multicolumn{4}{|l|}{\textbf{Triage Acuity Iter Category}} \\ \hline
\textit{ESI 3 - Urgent} & 2.19\% (5017) & 97.81\% (224123) &  \\ 
\textit{ESI 2 - Emergent} & 10.06\% (14229) & 89.94\% (127277) &  \\ 
\textit{ESI 4 - Less Urgent} & 0.05\% (13) & 99.95\% (28502) &  \\ 
\textit{ESI 1 - Resuscitation} & 35.89\% (8911) & 64.11\% (15915) &  \\ 
\textit{ESI 5 - Non-Urgent} & 0.00\% (0) & 100.00\% (1100) &  \\ 
\hline
\multicolumn{4}{|l|}{\textbf{Arrival Transport Walk In}} \\ \hline
\textit{False} & 13.18\% (22840) & 86.82\% (150398) &  \\ 
\textit{True} & 2.12\% (5330) & 97.88\% (246519) &  \\ 
\hline
\multicolumn{4}{|l|}{\textbf{Score News Category}} \\ \hline
\textit{Low Risk (1)} & 4.15\% (5084) &

In [12]:
# Generate LaTeX table for categorical variables
latex_table = """
\\begin{table}[h]
\\centering
\\renewcommand{\\arraystretch}{1.2}
\\setlength{\\tabcolsep}{3pt}
\\begin{threeparttable}
\\begin{tabular}{|p{3.0cm}|p{2.05cm}|p{2.05cm}|p{0.9cm}|}
\\hline
\\textbf{Variable} & \\textbf{Critical Outcome} & \\textbf{Non-Critical Outcome} & \\textbf{GB Importance} \\\\ 
\\hline
"""

current_variable = None
for _, row in df_summary.iterrows():
    variable, category, total_count, crit_count, crit_percent, non_crit_count, non_crit_percent = row

    # If new categorical variable, add header
    if variable != current_variable:
        if current_variable is not None:
            latex_table += "\\hline\n"  # Close previous section
        latex_table += f"\\multicolumn{{4}}{{|l|}}{{\\textbf{{{variable.replace('_', ' ').title()}}}}} \\\\ \\hline\n"
        current_variable = variable

    # Add category values
    latex_table += f"\\textit{{{category}}} & {crit_percent} ({crit_count}) & {non_crit_percent} ({non_crit_count}) &  \\\\ \n"

# Close table structure
latex_table += """\\hline
\\end{tabular}
\\begin{tablenotes}
\\item \\footnotesize All variables were statistically significant ($p<0.01$).
\\end{tablenotes}
\\end{threeparttable}
\\caption{Summary Statistics of Categorical Variables}
\\label{tab:summary_variables}
\\end{table}
"""

# Output the LaTeX table code
print(latex_table)



\begin{table}[h]
\centering
\renewcommand{\arraystretch}{1.2}
\setlength{\tabcolsep}{3pt}
\begin{threeparttable}
\begin{tabular}{|p{3.0cm}|p{2.05cm}|p{2.05cm}|p{0.9cm}|}
\hline
\textbf{Variable} & \textbf{Critical Outcome} & \textbf{Non-Critical Outcome} & \textbf{GB Importance} \\ 
\hline
\multicolumn{4}{|l|}{\textbf{Triage Acuity Iter Category}} \\ \hline
\textit{ESI 3 - Urgent} & 2.19% (5017) & 97.81% (224123) &  \\ 
\textit{ESI 2 - Emergent} & 10.06% (14229) & 89.94% (127277) &  \\ 
\textit{ESI 4 - Less Urgent} & 0.05% (13) & 99.95% (28502) &  \\ 
\textit{ESI 1 - Resuscitation} & 35.89% (8911) & 64.11% (15915) &  \\ 
\textit{ESI 5 - Non-Urgent} & 0.00% (0) & 100.00% (1100) &  \\ 
\hline
\multicolumn{4}{|l|}{\textbf{Arrival Transport Walk In}} \\ \hline
\textit{False} & 13.18% (22840) & 86.82% (150398) &  \\ 
\textit{True} & 2.12% (5330) & 97.88% (246519) &  \\ 
\hline
\multicolumn{4}{|l|}{\textbf{Score News Category}} \\ \hline
\textit{Low Risk (1)} & 4.15% (5084) & 95.85% (117423